In [1]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.optimize import minimize

In [2]:

# ------------------------------------------------------------------------------
# User's selection of recipe IDs (duplicates allowed)
# For example: [3, 1, 5, 2, 7, 3, 5]
# We will optimize over a permutation of the indices (0 to 6).
# ------------------------------------------------------------------------------
user_selection = np.array([3, 1, 5, 2, 7, 3, 5])

# ------------------------------------------------------------------------------
# Define processing times (5 machines x 7 recipes).
# Each column corresponds to a recipe ID (assumed 1-indexed).
# For example, processing_times[m, j] is the processing time on machine m for recipe (j+1).
# ------------------------------------------------------------------------------
processing_times = np.array([
    [10, 12,  9, 11, 10, 13,  8],   # Machine 1 times for recipes 1..7
    [ 9, 11, 10, 10,  9, 12,  7],    # Machine 2
    [ 8, 10, 11,  9,  8, 11,  7],    # Machine 3
    [11, 13, 10, 12, 11, 14,  9],    # Machine 4
    [10, 12,  9, 11, 10, 13,  8]     # Machine 5
])

# ------------------------------------------------------------------------------
# Define changeover times (7 recipes x 7 recipes).
# changeover_times[i, j] is the time to change from recipe (i+1) to recipe (j+1).
# Diagonals are zero (no changeover if the same recipe follows).
# ------------------------------------------------------------------------------
changeover_times = np.array([
    [0, 2, 3, 2, 1, 2, 3],
    [2, 0, 2, 3, 2, 3, 2],
    [3, 2, 0, 2, 3, 2, 2],
    [2, 3, 2, 0, 2, 3, 2],
    [1, 2, 3, 2, 0, 2, 3],
    [2, 3, 2, 3, 2, 0, 2],
    [3, 2, 2, 2, 3, 2, 0]
])

# ------------------------------------------------------------------------------
# Tact time (if needed) can be incorporated into processing times.
# For this example, we set tact_time to 0.
# ------------------------------------------------------------------------------
tact_time = 0

# ------------------------------------------------------------------------------

In [3]:
# ------------------------------------------------------------------------------
# Define the Bakery Scheduling Problem with no-wait and postponement.
# The candidate solution is a permutation of indices [0, 1, ..., n_jobs-1].
# ------------------------------------------------------------------------------
class BakerySchedulingMultisetProblem(Problem):
    def __init__(self, user_selection, processing_times, changeover_times, tact_time):
        self.user_selection = user_selection
        self.n_jobs = len(user_selection)
        self.n_machines = processing_times.shape[0]
        self.processing_times = processing_times
        self.changeover_times = changeover_times
        self.tact_time = tact_time
        # Decision variable: a permutation of indices from 0 to n_jobs-1.
        super().__init__(n_var=self.n_jobs, n_obj=1, xl=0, xu=self.n_jobs-1, type_var=int)
    
    def _evaluate(self, X, out, *args, **kwargs):
        F = []
        # X is an array where each row is a candidate permutation (of indices).
        for candidate in X:
            # Decode candidate: reorder the user_selection accordingly.
            sequence = self.user_selection[candidate]
            # Calculate makespan using our evaluation method that includes postponement.
            makespan = self.evaluate_sequence(sequence)
            F.append(makespan)
        out["F"] = np.column_stack(F)
    
    def evaluate_sequence(self, sequence):
        """
        Calculate the makespan of a given production sequence while enforcing
        a no-wait condition via a postponement (delay) on the first machine.
        
        For each job i in the sequence (i=0,1,...,n_jobs-1):
          - For i == 0, no postponement is needed (r[0] = 0).
          - For i >= 1, compute the minimum required start delay r[i] such that for each machine m (m >= 2)
            the following no-wait condition holds:
            
            r[i] + sum_{k=1}^{m} p_{k}(job_i) >= completion time of job (i-1) on machine (m-1) + changeover_time(job_{i-1}, job_i)
            
            Here, pₖ(job) is the processing time on machine k for the given job.
        
        Once r[i] is determined, schedule job i on machine 1 at time r[i] and then sequentially
        on subsequent machines without waiting.
        """
        n_jobs = len(sequence)
        M = self.n_machines
        # r[i]: postponement time for job i (start time on machine 1)
        r = np.zeros(n_jobs)
        # completion[m, i]: finish time of job i on machine m.
        completion = np.zeros((M, n_jobs))
        
        # --- Schedule the first job (i=0) ---
        idx = sequence[0] - 1  # adjust recipe ID to zero-index
        # For the first job, no postponement:
        completion[0, 0] = r[0] + self.processing_times[0, idx]
        for m in range(1, M):
            completion[m, 0] = completion[m-1, 0] + self.processing_times[m, idx]
        
        # --- Schedule subsequent jobs (i >= 1) ---
        for i in range(1, n_jobs):
            idx = sequence[i] - 1
            # Compute required postponement r[i] to avoid waiting on any machine.
            required_delay = 0
            # For each machine m from 2 to M, enforce:
            # r[i] >= (completion[m-1, i-1] + changeover_time) - (sum_{k=1}^{m} p_k(job_i))
            for m in range(1, M):  # m = 1 to M-1 corresponds to machine 2 to M.
                proc_sum = np.sum(self.processing_times[:m+1, idx])  # sum processing times from machine 1 to m+1
                prev_completion = completion[m-1, i-1]
                change_time = self.changeover_times[sequence[i-1]-1, idx]
                delay = prev_completion + change_time - proc_sum
                if delay > required_delay:
                    required_delay = delay
            r[i] = max(0, required_delay)  # postponement cannot be negative
            
            # Now schedule job i using its postponement r[i].
            completion[0, i] = r[i] + self.processing_times[0, idx]
            for m in range(1, M):
                # In a no-wait flow shop, once the job starts on machine 1 at time r[i],
                # it flows continuously through subsequent machines.
                completion[m, i] = completion[m-1, i] + self.processing_times[m, idx]
        
        # The makespan is the finish time of the last job on the last machine.
        return completion[-1, -1]

In [4]:

# ------------------------------------------------------------------------------
# Create an instance of the problem.
# ------------------------------------------------------------------------------
problem = BakerySchedulingMultisetProblem(user_selection, processing_times, changeover_times, tact_time)

# ------------------------------------------------------------------------------
# Set up the Genetic Algorithm using pymoo's permutation operators.
# ------------------------------------------------------------------------------
algorithm = GA(
    pop_size=50,
    sampling=PermutationRandomSampling(),
    crossover=OrderCrossover(),
    mutation=InversionMutation(),
    eliminate_duplicates=True
)

# ------------------------------------------------------------------------------
# Run the optimization (here for 100 generations).
# ------------------------------------------------------------------------------
res = minimize(
    problem,
    algorithm,
    termination=("n_gen", 100),
    seed=42,
    verbose=True
)

# ------------------------------------------------------------------------------
# Decode and print the best solution.
# ------------------------------------------------------------------------------
best_permutation = res.X   # a permutation of indices
best_sequence = user_selection[best_permutation]  # decoded production sequence
print("Best permutation (indices):", best_permutation)
print("Best production sequence:", best_sequence)
print("Best makespan (minutes):", res.F[0])

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       50 |  4.872000E+01 |  4.100000E+01
     2 |      100 |  4.600000E+01 |  4.100000E+01
     3 |      150 |  4.410000E+01 |  4.100000E+01
     4 |      200 |  4.122000E+01 |  4.100000E+01
     5 |      250 |  4.100000E+01 |  4.100000E+01
     6 |      300 |  4.100000E+01 |  4.100000E+01
     7 |      350 |  4.100000E+01 |  4.100000E+01
     8 |      400 |  4.100000E+01 |  4.100000E+01
     9 |      450 |  4.100000E+01 |  4.100000E+01
    10 |      500 |  4.100000E+01 |  4.100000E+01
    11 |      550 |  4.100000E+01 |  4.100000E+01
    12 |      600 |  4.100000E+01 |  4.100000E+01
    13 |      650 |  4.100000E+01 |  4.100000E+01
    14 |      700 |  4.100000E+01 |  4.100000E+01
    15 |      750 |  4.100000E+01 |  4.100000E+01
    16 |      800 |  4.100000E+01 |  4.100000E+01
    17 |      850 |  4.100000E+01 |  4.100000E+01
    18 |      900 |  4.100000E+01 |  4.100000E+01
    19 |      950 |  4.100000E+01 |  4.100000E+01
